In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
print(logits)
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[-0.1295,  0.1392, -0.0207,  0.0483,  0.0079, -0.0302, -0.0006,  0.0527,
          0.1070, -0.0710]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[0.0867, 0.1134, 0.0967, 0.1036, 0.0995, 0.0957, 0.0986, 0.1040, 0.1098,
         0.0919]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([1], device='cuda:0')


In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")


Before ReLU: tensor([[-0.2243,  0.0126,  0.3192,  0.0985, -0.4421, -0.3991, -0.2618,  0.3113,
         -0.0502, -0.2663,  0.0125,  0.3046, -0.1407,  0.1058,  0.0710,  0.1513,
         -0.0660,  0.1292,  0.3765, -0.2206],
        [-0.1289,  0.1966,  0.0271,  0.1455, -0.4503, -0.2753, -0.3212,  0.2666,
         -0.0320,  0.0526,  0.4484, -0.0804,  0.0493,  0.0543,  0.1296,  0.0015,
          0.0757, -0.0708,  0.3247, -0.0657],
        [-0.2434, -0.2126,  0.0563, -0.1315, -0.3312, -0.1519, -0.1861,  0.0867,
          0.2514, -0.0222,  0.2907,  0.0290, -0.0713,  0.0386,  0.0337,  0.0386,
          0.0148, -0.1130, -0.0411, -0.0464]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0126, 0.3192, 0.0985, 0.0000, 0.0000, 0.0000, 0.3113, 0.0000,
         0.0000, 0.0125, 0.3046, 0.0000, 0.1058, 0.0710, 0.1513, 0.0000, 0.1292,
         0.3765, 0.0000],
        [0.0000, 0.1966, 0.0271, 0.1455, 0.0000, 0.0000, 0.0000, 0.2666, 0.0000,
         0.0526, 0.4484, 0.0000, 0.0493, 0.0543, 0.12

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0011, -0.0100, -0.0243,  ..., -0.0351, -0.0143,  0.0080],
        [-0.0223, -0.0259, -0.0150,  ..., -0.0155,  0.0147,  0.0048]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0182, -0.0006], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0121,  0.0363, -0.0424,  ..., -0.0380, -0.0213, -0.0126],
        [-0.0187, -0.0020, -0.0037,  ...,  0.0188, -0.0438,  0.0202]],
       device='cuda:0', grad_fn=<Sl